<a href="https://colab.research.google.com/github/van-dang/MRI-Cloud/blob/master/ECS_226Cylinders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the code

In [0]:
# This program solves the Bloch-Torrey equation applied to computational diffusion MRI using 
# the finite element method coupled with the theta-method for the spatial discretization.

# The scope of usage: 
# (1) Single domains, Multilayered structures, manifolds
# (2) Membrane permeability for internal interfaces
#     Artificial permeability at the external interfaces
# (3) pure homogeneous Neumann BCs, (4) pseudo-periodic BCs

# Copyright (C) 2019 Van-Dang Nguyen (vdnguyen@kth.se)

# This file is part of DOLFIN.

# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.

# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.

# First added:  2017-10-10
# Last changed: 2019-04-25

# This demo is maintained by Van-Dang Nguyen
# Please report possible problems to vdnguyen@kth.se

# Setting a working environment with FEniCS

In [2]:
try:
    from google.colab import files
except:
    print("This is not google.colab")
    
import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *

gmsh_dir=!which gmsh
if len(gmsh_dir)==0:
  !apt-get install gmsh
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

clear_output()

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

# Disable warnings
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("rothemain.rothe_utils")
logging.getLogger('UFL').setLevel(logging.WARNING)
logging.getLogger('FFC').setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore")


dolfin version: 2019.1.0


#Load pre-defined functions

In [3]:
import sympy as sp  
exists = os.path.isfile('DmriFemBuitInFunc.py')
isupdate = False
if (exists==False or isupdate==True):
    if isupdate==True:
        !rm DmriFemBuitInFunc.py
    print("Load pre-defined functions from GitHub")
    !wget --quiet https://raw.githubusercontent.com/van-dang/MRI-Cloud/master/DmriFemBuitInFunc.py
from DmriFemBuitInFunc import *
      

Load pre-defined functions from GitHub


# Working on the mesh and submesh

In [4]:
is_mesh_file_exist = os.path.isfile('mesh_226cylinders.xml.zip')
if is_mesh_file_exist==False:
    !wget --quiet https://github.com/van-dang/MRI-Cloud/raw/mesh/mesh_226cylinders.xml.zip
    !wget --quiet https://github.com/van-dang/MRI-Cloud/raw/mesh/submesh_226cylinders.xml.zip
!sudo apt-get install unzip
!rm -rf mesh_226cylinders.xml submesh_226cylinders.xml __MACOSX
!unzip mesh_226cylinders.xml.zip
!unzip submesh_226cylinders.xml.zip

is_partition_function_exist = os.path.isfile('GetPartitionMarkers.py')
if is_partition_function_exist==False:
    !wget --quiet https://raw.githubusercontent.com/van-dang/MRI-Cloud/mesh/GetPartitionMarkers.py

mymesh = Mesh("mesh_226cylinders.xml");  
cmpt_mesh = Mesh('submesh_226cylinders.xml')
phase, partion_list, partition_marker = CreatePhaseFunc(mymesh, [], [cmpt_mesh], None)

File("Phase.pvd")<<phase


Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.
Archive:  mesh_226cylinders.xml.zip
  inflating: mesh_226cylinders.xml   
   creating: __MACOSX/
  inflating: __MACOSX/._mesh_226cylinders.xml  
Archive:  submesh_226cylinders.xml.zip
  inflating: submesh_226cylinders.xml  
  inflating: __MACOSX/._submesh_226cylinders.xml  


# Solve the Bloch-Torrey equation

In [0]:
class MRI_simulation():
    def __init__(self):
          self.nskip = 5;    # Output frequency (for visualization only)
          self.theta = 0.5;  # theta=0.5: midpoint method

    def InitialCondition(self, mydomain, Dirac_Delta):
          if Dirac_Delta==None:
              if mydomain.gdim==2:
                  Dirac_Delta = Expression("x[0]*x[0]+x[1]*x[1]<eps",eps=1e6, domain=mydomain.mymesh, degree=1);
              if mydomain.gdim==3:
                  Dirac_Delta = Expression("x[0]*x[0]+x[1]*x[1]+x[2]*x[2]<eps",eps=1e6, domain=mydomain.mymesh, degree=1);
              Dirac_Delta = interpolate(Dirac_Delta, mydomain.V);
          u_0 = Function(mydomain.W);
          assign(u_0.sub(0), Dirac_Delta)
          if (mydomain.IsDomainMultiple==True):
              assign(u_0.sub(2), Dirac_Delta)  
          return Dirac_Delta, u_0
        
    def solve(self, mydomain, mri_para, linsolver, ic=None): 

          self.Dirac_Delta, self.u_0 = self.InitialCondition(mydomain, ic)

          stepcounter = 0;

          M = MassMatrix(mydomain);

          tp = 0;
          self.t = tp;

          comm = MPI.comm_world
          rank = comm.Get_rank()

          start_time = time.time()

          while self.t < mri_para.T + self.k: # Time-stepping loop                                                                                                                                
              if stepcounter % self.nskip == 0 and rank==0:
                  print('t: %6.2f '%self.t, 'T: %6.2f'%mri_para.T, 'dt: %.1f'%self.k,'qvalue: %e'%mri_para.qvalue,'Completed %3.2f%%'%(float(self.t)/float(mri_para.T+self.k)*100.0));
              # tm = tp + 0.5*self.k;
              ft = mri_para.time_profile(self.t);
              ift = mri_para.itime_profile(self.t);
              ft_p = mri_para.time_profile(tp);
              ift_p = mri_para.itime_profile(tp);
              
              L = ThetaMethodL(ft_p, ift_p, mri_para, self, mydomain);
              A = 1/self.k*M + assemble(ThetaMethodF(ft, ift, mri_para, self, mydomain))

              b = assemble(L);
              linsolver.solve(A, self.u_0.vector(),b);

              tp = self.t;
              self.t += self.k;
              stepcounter += 1;
 
          self.elapsed_time = time.time() - start_time
          if rank==0:
              print("Successfully Completed! Elapsed time: %f seconds"%self.elapsed_time)

        
mri_simu = MRI_simulation()
mri_para = MRI_parameters()

bvalues = [10000]

for bvalue in bvalues:
    #################################################################################
    #########################  Pre-defined parameters ###############################
    mri_para.bvalue = bvalue                             # bvalue
    mri_para.delta, mri_para.Delta = 10000, 13000        # time sequence
    mri_para.set_gradient_dir(mymesh, 1, 1, 0)           # gradient direction
    mri_para.T = mri_para.Delta+mri_para.delta
    mri_para.fs_sym = sp.Piecewise(
                    (  1., mri_para.s < mri_para.delta ),
                    (  0., mri_para.s < mri_para.Delta ),
                    ( -1., mri_para.s < mri_para.T ),
                    (  0., True )  
                ) 

    mri_para.Apply()
    mri_simu.k = 200;                                    # time-step size
    mri_simu.nskip = 1;                                  # frequency to print ouputs

    mydomain = MyDomain(mymesh, mri_para)
    mydomain.phase = phase
    mydomain.PeriodicDir = [0, 0, 0];            # Direction of the periodicity
    mydomain.IsDomainPeriodic = False            # Confirm if the mesh if periodic
    mydomain.IsDomainMultiple = True             # Confirm if the mesh is multiple
    mydomain.kappa = 1e-5                        # Permeability
    ################################################################################

    mydomain.Apply()   # Call Apply before setting the diffusion tensor

    ################################################################################
    # Impose the diffusion coefficient
    D0_array=[2e-3, 2e-3]

    # Variable tensor
    dofmap_DG = mydomain.V_DG.dofmap()
    d00 = Function(mydomain.V_DG); d01 = Function(mydomain.V_DG); d02 = Function(mydomain.V_DG)
    d10 = Function(mydomain.V_DG); d11 = Function(mydomain.V_DG); d12 = Function(mydomain.V_DG)
    d20 = Function(mydomain.V_DG); d21 = Function(mydomain.V_DG); d22 = Function(mydomain.V_DG)

    for cell in cells(mymesh):
        p = cell.midpoint() # the coordinate of the cell center.
        cmk = partition_marker[cell.index()]
        cell_dof = dofmap_DG.cell_dofs(cell.index())
        d00.vector()[cell_dof] = D0_array[cmk]; 
        d11.vector()[cell_dof] = D0_array[cmk]; 
        d22.vector()[cell_dof] = D0_array[cmk];

    mydomain.ImposeDiffusionTensor(d00, d01, d02, d10, d11, d12, d20, d21, d22)
    #################################################################################
    #################################################################################

    linsolver = KrylovSolver("bicgstab","petsc_amg")
    linsolver.parameters["absolute_tolerance"] = 1e-4
    linsolver.parameters["relative_tolerance"] = 1e-4
    linsolver.parameters["maximum_iterations"] = 10000

    mri_simu.solve(mydomain, mri_para, linsolver)

    ctext = ''
    Post_processing(mydomain, mri_para, mri_simu, plt, ctext)


Domain size: xmin=-77.088005, ymin=-75.888884, zmin=-1.000000, xmax=77.088005, ymax=75.888884, zmax=1.000000
Function Space for Two-compartment Domains has 4 components
(ur0, ui0, ur1, ur1): r-real, i-imaginary
Initialize a standard function space.
Impose Diffusion Tensor ...
t:   0.00  T: 23000.00 dt: 100.0 qvalue: 1.017095e-04 Completed 0.00%
